### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

### Importing function

In [73]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

### Extracting raw data

#### Bicimad

In [43]:
bicimad_stations=pd.read_csv('../data/bicimad_stations.csv')

In [44]:
bicimad_stations=bicimad_stations[["address","geometry_coordinates","name"]]

In [45]:
bicimad_stations[["long_finish","lat_finish"]]=bicimad_stations['geometry_coordinates'].str.split(',', 1, expand=True)

In [46]:
bicimad_stations["lat_finish"]=bicimad_stations["lat_finish"].str.replace("]","", regex=True)

In [47]:
bicimad_stations["long_finish"]=bicimad_stations["long_finish"].str.replace("[","", regex=True)

In [48]:
bicimad_stations = bicimad_stations.rename(columns={'address': 'Station location', 'name': 'BiciMAD station'})

In [49]:
bicimad_stations = bicimad_stations.drop(columns='geometry_coordinates')

In [50]:
bicimad_stations

,Station location,BiciMAD station,long_finish,lat_finish
0,Puerta del Sol nº 1,Puerta del Sol A,-3.7018341,40.4172137
1,Puerta del Sol nº 1,Puerta del Sol B,-3.701602938060457,40.41731271011562
2,Calle Miguel Moya nº 1,Miguel Moya,-3.7058415,40.4205886
3,Plaza del Conde del Valle de Súchil nº 3,Plaza Conde Suchil,-3.7069171,40.4302937
4,Calle Manuela Malasaña nº 5,Malasaña,-3.7025875,40.4285524
...,...,...,...,...
259,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 1,-3.72699,40.44375
260,Avenida de la Complutense (Metro Ciudad Univer...,Ciudad Universitaria 2,-3.72693,40.44342
261,Calle José Antonio Novais frente al nº 12,Facultad Biología,-3.7272945,40.4483322
262,Avenida Complutense nº 23,Facultad Derecho,-3.72937,40.45109


#### Interesting Points

In [34]:
interest_points=pd.read_csv('../data/300356-0-monumentos-ciudad-madrid.csv', sep=';',encoding = "ISO-8859-1")

In [35]:
interest_points=interest_points[["NOMBRE","NOMBRE-VIA","CLASE-VIAL","NUM","LATITUD","LONGITUD"]]

In [37]:
interest_points = interest_points.rename(columns=
{'NOMBRE': 'Place of interest', 'NOMBRE-VIA': 'Place address Name', 'CLASE-VIAL': 'Place address Class'
,'NUM': 'Place address Num','LATITUD': 'lat_start','LONGITUD': 'long_start'})

In [52]:
interest_points["Type of place"] = "Monumentos de la ciudad de Madrid" 

In [53]:
interest_points

,Place of interest,Place address Name,Place address Class,Place address Num,lat_start,long_start,Type of place
0,A las víctimas del Holocausto,TRES CULTURAS,JARDIN,NaN,40.464322,-3.603558,Monumentos de la ciudad de Madrid
1,A los Abuelos,Alicún,C,NaN,40.479812,-3.661719,Monumentos de la ciudad de Madrid
2,A Neruda-El Ser alado,José Luis Ozores,Gta,116,40.381855,-3.652419,Monumentos de la ciudad de Madrid
3,Abogados Laboristas,Atocha,C,55,40.412911,-3.700017,Monumentos de la ciudad de Madrid
4,Abstracta I,NaN,NaN,NaN,40.421746,-3.662293,Monumentos de la ciudad de Madrid
...,...,...,...,...,...,...,...
1843,XXV aniversario Colegio Mayor Argentino,Martín Fierro,C,NaN,40.436127,-3.731997,Monumentos de la ciudad de Madrid
1844,XXV aniversario Colegio Mayor Argentino,Martín Fierro,C,NaN,40.436100,-3.732009,Monumentos de la ciudad de Madrid
1845,Yawar Fiesta,NaN,NaN,NaN,NaN,NaN,Monumentos de la ciudad de Madrid
1846,'Sin título',NaN,NaN,NaN,NaN,NaN,Monumentos de la ciudad de Madrid


### Joining Data

In [55]:
interest_points_bicimad = bicimad_stations.assign(key=0).merge(interest_points.assign(key=0), how='left', on = 'key')

In [56]:
interest_points_bicimad

,Station location,BiciMAD station,long_finish,lat_finish,key,Place of interest,Place address Name,Place address Class,Place address Num,lat_start,long_start,Type of place
0,Puerta del Sol nº 1,Puerta del Sol A,-3.7018341,40.4172137,0,A las víctimas del Holocausto,TRES CULTURAS,JARDIN,NaN,40.464322,-3.603558,Monumentos de la ciudad de Madrid
1,Puerta del Sol nº 1,Puerta del Sol A,-3.7018341,40.4172137,0,A los Abuelos,Alicún,C,NaN,40.479812,-3.661719,Monumentos de la ciudad de Madrid
2,Puerta del Sol nº 1,Puerta del Sol A,-3.7018341,40.4172137,0,A Neruda-El Ser alado,José Luis Ozores,Gta,116,40.381855,-3.652419,Monumentos de la ciudad de Madrid
3,Puerta del Sol nº 1,Puerta del Sol A,-3.7018341,40.4172137,0,Abogados Laboristas,Atocha,C,55,40.412911,-3.700017,Monumentos de la ciudad de Madrid
4,Puerta del Sol nº 1,Puerta del Sol A,-3.7018341,40.4172137,0,Abstracta I,NaN,NaN,NaN,40.421746,-3.662293,Monumentos de la ciudad de Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...
487867,Calle Zurbano nº 90,Zurbano,-3.69281,40.43837,0,XXV aniversario Colegio Mayor Argentino,Martín Fierro,C,NaN,40.436127,-3.731997,Monumentos de la ciudad de Madrid
487868,Calle Zurbano nº 90,Zurbano,-3.69281,40.43837,0,XXV aniversario Colegio Mayor Argentino,Martín Fierro,C,NaN,40.436100,-3.732009,Monumentos de la ciudad de Madrid
487869,Calle Zurbano nº 90,Zurbano,-3.69281,40.43837,0,Yawar Fiesta,NaN,NaN,NaN,NaN,NaN,Monumentos de la ciudad de Madrid
487870,Calle Zurbano nº 90,Zurbano,-3.69281,40.43837,0,'Sin título',NaN,NaN,NaN,NaN,NaN,Monumentos de la ciudad de Madrid


In [92]:
interest_points_bicimad[["long_finish", "lat_finish","lat_start","long_start"]] = interest_points_bicimad[["long_finish", "lat_finish","lat_start","long_start"]].apply(pd.to_numeric)

In [105]:
interest_points_test= interest_points_bicimad[(interest_points_bicimad["BiciMAD station"]
=="Puerta del Sol A") & (interest_points_bicimad["Place of interest"]=="A las víctimas del Holocausto")]

In [108]:
interest_points_bicimad['distance'] =interest_points_bicimad.apply(lambda x: distance_meters(x['lat_start'], x['long_start'],x['lat_finish'], x['long_finish']), axis=1)


KeyboardInterrupt: 

In [101]:
distance_meters(40.464322,-3.603558,40.4172137,-3.7018341)

0    12152.064218
dtype: float64

In [107]:
interest_points_test

,Station location,BiciMAD station,long_finish,lat_finish,key,Place of interest,Place address Name,Place address Class,Place address Num,lat_start,long_start,Type of place,distance
0,Puerta del Sol nº 1,Puerta del Sol A,-3.701834,40.417214,0,A las víctimas del Holocausto,TRES CULTURAS,JARDIN,NaN,40.464322,-3.603558,Monumentos de la ciudad de Madrid,12152.077352
